In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
df_sentiment = pd.read_csv('./data/sentiment_data.csv', index_col='date')
df_regime = pd.read_csv('./data/pivot_regime.csv', skiprows=1, index_col=0, parse_dates=[0])
df_returns = pd.read_csv('./data/pivot_returns.csv', skiprows=1, index_col=0, parse_dates=[0])

C:\Users\ROG\AppData\Local\Temp\ipykernel_3548\219081687.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_regime = pd.read_csv('./data/pivot_regime.csv', skiprows=1, index_col=0, parse_dates=[0])
C:\Users\ROG\AppData\Local\Temp\ipykernel_3548\219081687.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_returns = pd.read_csv('./data/pivot_returns.csv', skiprows=1, index_col=0, parse_dates=[0])


In [3]:
df_sentiment.dropna(inplace=True)
df_regime.dropna(inplace=True)
df_returns.dropna(inplace=True)

df_regime.index.name = 'Date'
df_returns.index.name = 'Date'

In [4]:
df_sentiment

,AAPL,AMZN,GOOG,MSFT,NFLX
date,,,,,
2025-04-03,0.108453,0.152113,0.250044,0.086048,0.213562
2025-04-04,0.108453,0.152113,0.250044,0.086048,0.321464
2025-04-05,0.108453,0.152113,0.250044,0.086048,0.215420
2025-04-06,0.108453,0.152113,0.250044,0.086048,0.280937
2025-04-07,0.108453,0.152113,0.250044,0.086048,0.262796
2025-04-08,0.108453,0.152113,0.250044,0.086048,0.315149
2025-04-09,0.108453,0.152113,0.250044,0.086048,0.011468
2025-04-10,0.108453,0.152113,0.250044,0.086048,0.153189
2025-04-11,0.046556,0.179771,0.250044,0.207683,0.147609


In [5]:
target_length = df_regime.shape[0]

In [6]:
interpolated_sentiment = pd.DataFrame()

for stock in df_sentiment.columns:
    original = df_sentiment[stock].values
    x_old = np.arange(len(original))
    x_new = np.linspace(0, len(original)-1, target_length)

    interpolated_values = np.interp(x_new, x_old, original)

    scaler = MinMaxScaler()
    scaled_values = scaler.fit_transform(interpolated_values.reshape(-1, 1)).flatten()

    interpolated_sentiment[stock] = scaled_values

interpolated_sentiment.index = pd.date_range(
    start = df_returns.index.min(), periods=target_length, freq='B'
)

In [7]:
interpolated_sentiment.head()

,AAPL,AMZN,GOOG,MSFT,NFLX
2015-01-09,0.653242,0.017892,1.0,0.0,0.651971
2015-01-12,0.653242,0.017892,1.0,0.0,0.653497
2015-01-13,0.653242,0.017892,1.0,0.0,0.655023
2015-01-14,0.653242,0.017892,1.0,0.0,0.656550
2015-01-15,0.653242,0.017892,1.0,0.0,0.658076


In [8]:
df_returns.head()

,AAPL,AMZN,GOOG,MSFT,NFLX
Date,,,,,
2015-01-09,0.001072,-0.011749,-0.012951,-0.008405,-0.015458
2015-01-12,-0.024641,-0.018590,-0.007296,-0.012502,-0.031765
2015-01-13,0.008879,0.011427,0.007370,-0.005150,0.015557
2015-01-14,-0.003811,-0.004987,0.009452,-0.008628,0.001390
2015-01-15,-0.027140,-0.021550,0.001837,-0.010444,-0.001480


In [9]:
# Ensure both DataFrames have datetime index
df_returns.index = pd.to_datetime(df_returns.index)
interpolated_sentiment.index = pd.to_datetime(interpolated_sentiment.index)

# Reindex sentiment to match returns (forward-fill or interpolate)
interpolated_sentiment = interpolated_sentiment.reindex(df_returns.index, method='ffill')

In [10]:
sentiment_weighted_df = df_returns * interpolated_sentiment
sentiment_weighted_df.head()

,AAPL,AMZN,GOOG,MSFT,NFLX
Date,,,,,
2015-01-09,0.000701,-0.000210,-0.012951,-0.0,-0.010078
2015-01-12,-0.016096,-0.000333,-0.007296,-0.0,-0.020759
2015-01-13,0.005800,0.000204,0.007370,-0.0,0.010190
2015-01-14,-0.002489,-0.000089,0.009452,-0.0,0.000912
2015-01-15,-0.017729,-0.000386,0.001837,-0.0,-0.000974


In [11]:
df_regime

,AAPL,AMZN,GOOG,MSFT,NFLX
Date,,,,,
2015-01-09,Neutral,Bearish,Neutral,Neutral,Bearish
2015-01-12,Neutral,Bearish,Bullish,Neutral,Bearish
2015-01-13,Neutral,Bullish,Bullish,Neutral,Bearish
2015-01-14,Neutral,Bearish,Bullish,Neutral,Bearish
2015-01-15,Neutral,Bearish,Bullish,Bullish,Bearish
...,...,...,...,...,...
2024-12-24,Bearish,Neutral,Neutral,Neutral,Neutral
2024-12-26,Bearish,Bearish,Neutral,Bullish,Neutral
2024-12-27,Bearish,Bearish,Neutral,Bullish,Neutral


In [12]:
bullish_returns = sentiment_weighted_df.copy()

for stock in sentiment_weighted_df.columns:
    for ind in df_regime.index:
        if df_regime.loc[ind, stock] != 'Bearish':
            bullish_returns.loc[ind, stock] = 0.0

In [13]:
bullish_returns

,AAPL,AMZN,GOOG,MSFT,NFLX
Date,,,,,
2015-01-09,0.000000,-0.000210,0.0,0.0,-0.010078
2015-01-12,0.000000,-0.000333,0.0,0.0,-0.020759
2015-01-13,0.000000,0.000000,0.0,0.0,0.010190
2015-01-14,0.000000,-0.000089,0.0,0.0,0.000912
2015-01-15,0.000000,-0.000386,0.0,0.0,-0.000974
...,...,...,...,...,...
2024-12-24,0.009855,0.000000,0.0,0.0,0.000000
2024-12-26,0.002727,-0.008732,0.0,0.0,0.000000
2024-12-27,-0.011369,-0.014534,0.0,0.0,0.000000


## HRP Implementation  

In [14]:
# Correltaion matrix -> distance matrix
def correl_dist(corr):
    return ((1 - corr) / 2.) ** 0.5

In [15]:
# Compute inverse portfolio weights
def get_ivp(cov):
    ivp = 1. / np.diag(cov)
    ivp /= ivp.sum()
    return ivp

In [16]:
# Compute total variance of a given cluster
def get_cluster_var(cov, cluster_items):
    cov_ = cov.loc[cluster_items, cluster_items]
    w_ = get_ivp(cov_).reshape(-1, 1)
    return np.dot(np.dot(w_.T, cov_), w_)[0, 0]

In [17]:
# Reorder asset in quasi-diagonal form after hierarchical clustering
def get_quasi_diag(link):
    link = link.astype(int)
    sort_ix = pd.Series([link[-1, 0], link[-1, 1]])
    num_items = link[-1, 3]
    while sort_ix.max() >= num_items:
        sort_ix.index = range(0, sort_ix.shape[0] * 2, 2)
        df0 = sort_ix[sort_ix >= num_items]
        i = df0.index
        j = df0.values - num_items
        sort_ix[i] = link[j, 0]
        df1 = pd.Series(link[j, 1], index=i + 1)
        sort_ix = pd.concat([sort_ix, df1])
        sort_ix = sort_ix.sort_index()
    return sort_ix.tolist()


In [18]:
# Implement the full HRP algorithm
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.spatial.distance import squareform

def get_hrp_weights(cov, corr):
    dist = correl_dist(corr)
    link = linkage(squareform(dist), 'single')
    sort_ix = get_quasi_diag(link)
    sorted_tickers = corr.index[sort_ix].tolist()

    hrp = pd.Series(1, index=sorted_tickers)
    clustered_items = [sorted_tickers]
    while len(clustered_items) > 0:
        clustered_items = [i[j:k] for i in clustered_items for j, k in ((0, len(i) // 2), (len(i) // 2, len(i))) if len(i) > 1]
        for i in range(0, len(clustered_items), 2):
            c_items0 = clustered_items[i]
            c_items1 = clustered_items[i + 1]
            var0 = get_cluster_var(cov, c_items0)
            var1 = get_cluster_var(cov, c_items1)
            alpha = 1 - var0 / (var0 + var1)
            hrp[c_items0] *= alpha
            hrp[c_items1] *= 1 - alpha
    return hrp / hrp.sum()

In [19]:
# Compute HRP weights on filtered returns
cov_matrix = bullish_returns.cov()
corr_matrix = bullish_returns.corr()


In [20]:
weights = get_hrp_weights(cov_matrix, corr_matrix)

C:\Users\ROG\AppData\Local\Temp\ipykernel_3548\2190638543.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.4044692 0.4044692]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  hrp[c_items0] *= alpha


In [21]:
print(f'HRP Portfolio weights (Bullish regime + Sentiment Weighted): \n{weights.sort_values(ascending=False)}')

HRP Portfolio weights (Bullish regime + Sentiment Weighted): 
MSFT    0.538904
AMZN    0.312661
AAPL    0.091808
GOOG    0.029073
NFLX    0.027554
dtype: float64
